In [25]:
import pandas as pd
import sqlite3


In [26]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [27]:
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [28]:
conn = sqlite3.connect("restaurants.db")


In [29]:


pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)


,name


In [30]:
final_dataset = pd.merge(
    orders,
    users,
    how="left",
    on="user_id"
)

final_dataset.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular


In [34]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=final_dataset)

MessageError: Error: credential propagation was unsuccessful

In [31]:
final_dataset.shape


(10000, 9)

In [32]:
final_dataset.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name', 'name', 'city', 'membership'],
      dtype='object')

In [33]:
final_dataset.to_csv(
    "final_food_delivery_dataset.csv",
    index=False
)


In [35]:
gold_members = final_dataset[final_dataset['membership'] == 'Gold']


In [36]:
city_revenue = gold_members.groupby('city')['total_amount'].sum().reset_index()
city_revenue = city_revenue.sort_values(by='total_amount', ascending=False)
city_revenue


,city,total_amount
1,Chennai,1080909.79
3,Pune,1003012.32
0,Bangalore,994702.59
2,Hyderabad,896740.19


In [37]:
top_city = city_revenue.iloc[0]['city']
top_revenue = city_revenue.iloc[0]['total_amount']

print(f"City with highest revenue from Gold members: {top_city} ({top_revenue})")


City with highest revenue from Gold members: Chennai (1080909.79)


In [41]:
user_total = final_dataset.groupby('user_id')['total_amount'].sum().reset_index()
user_total.head()


,user_id,total_amount
0,1,1289.66
1,2,7564.12
2,3,1839.51
3,4,3741.16
4,5,5742.88


In [42]:
high_value_users = user_total[user_total['total_amount'] > 1000]
high_value_users.shape[0]


2544

In [44]:
['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name', 'name', 'city', 'membership']


['order_id',
 'user_id',
 'restaurant_id',
 'order_date',
 'total_amount',
 'restaurant_name',
 'name',
 'city',
 'membership']

In [47]:
gold_members = final_dataset[final_dataset['membership'] == 'Gold']


In [48]:
avg_order_city = gold_members.groupby('city')['total_amount'].mean().reset_index()
avg_order_city = avg_order_city.sort_values(by='total_amount', ascending=False)
avg_order_city


,city,total_amount
1,Chennai,808.459080
2,Hyderabad,806.421034
0,Bangalore,793.223756
3,Pune,781.162243


In [49]:
top_city_avg = avg_order_city.iloc[0]['city']
top_avg_value = avg_order_city.iloc[0]['total_amount']

print(f"City with highest average order value among Gold members: {top_city_avg} (₹{top_avg_value:.2f})")


City with highest average order value among Gold members: Chennai (₹808.46)


In [50]:
# Add cuisine column if not already done
restaurant_cuisine = {
    'Tandoori Delight': 'Indian',
    'Dragon Wok': 'Chinese',
    'Pasta House': 'Italian',
    'Mexicana': 'Mexican'
    # Add all your restaurants here
}

final_dataset['cuisine_type'] = final_dataset['restaurant_name'].map(restaurant_cuisine)


In [51]:
restaurants_per_cuisine = final_dataset.groupby('cuisine_type')['restaurant_name'].nunique().reset_index()
restaurants_per_cuisine = restaurants_per_cuisine.rename(columns={'restaurant_name': 'num_restaurants'})
restaurants_per_cuisine


,cuisine_type,num_restaurants


In [52]:
revenue_per_cuisine = final_dataset.groupby('cuisine_type')['total_amount'].sum().reset_index()
revenue_per_cuisine = revenue_per_cuisine.rename(columns={'total_amount': 'total_revenue'})
revenue_per_cuisine


,cuisine_type,total_revenue


In [53]:
cuisine_summary = pd.merge(restaurants_per_cuisine, revenue_per_cuisine, on='cuisine_type')
cuisine_summary


,cuisine_type,num_restaurants,total_revenue


In [56]:
total_orders = final_dataset.shape[0]  # Total number of rows/orders


In [57]:
gold_orders = final_dataset[final_dataset['membership'] == 'Gold'].shape[0]


In [59]:
percent_gold = round((gold_orders / total_orders) * 100)
print(f"Percentage of orders placed by Gold members: {percent_gold}%")


Percentage of orders placed by Gold members: 50%


In [60]:
restaurant_stats = final_dataset.groupby('restaurant_name').agg(
    total_orders=('order_id', 'count'),
    avg_order_value=('total_amount', 'mean')
).reset_index()


In [61]:
small_restaurants = restaurant_stats[restaurant_stats['total_orders'] < 20]


In [62]:
top_restaurant = small_restaurants.sort_values(by='avg_order_value', ascending=False).iloc[0]

print(f"Restaurant: {top_restaurant['restaurant_name']}")
print(f"Average order value: ₹{top_restaurant['avg_order_value']:.2f}")
print(f"Total orders: {top_restaurant['total_orders']}")


Restaurant: Hotel Dhaba Multicuisine
Average order value: ₹1040.22
Total orders: 13


In [63]:
restaurant_cuisine = {
    'Tandoori Delight': 'Indian',
    'Dragon Wok': 'Chinese',
    'Pasta House': 'Italian',
    'Mexicana': 'Mexican'
    # Add all your restaurants here
}

final_dataset['cuisine_type'] = final_dataset['restaurant_name'].map(restaurant_cuisine)


In [64]:
combo_revenue = final_dataset.groupby(['membership', 'cuisine_type'])['total_amount'].sum().reset_index()
combo_revenue = combo_revenue.sort_values(by='total_amount', ascending=False)
combo_revenue


,membership,cuisine_type,total_amount


In [67]:
# Check which restaurants are missing in mapping
missing_restaurants = final_dataset.loc[final_dataset['restaurant_name'].isna() |
                                       ~final_dataset['restaurant_name'].isin(restaurant_cuisine.keys()),
                                       'restaurant_name'].unique()
print("Restaurants missing cuisine mapping:", missing_restaurants)


Restaurants missing cuisine mapping: ['New Foods Chinese' 'Ruchi Curry House Multicuisine'
 'Spice Kitchen Punjabi' 'Darbar Kitchen Non-Veg'
 'Royal Eatery South Indian' 'Annapurna Tiffins South Indian'
 'Royal Biryani North Indian' 'Spice Mess Punjabi' 'Ruchi Biryani Punjabi'
 'Taste of Biryani Non-Veg' 'Amma Delights Family Restaurant'
 'Royal Tiffins Multicuisine' 'Amma Tiffins South Indian'
 'Grand Cafe Punjabi' 'Amma Biryani North Indian'
 'Amma Restaurant South Indian' 'Ruchi Foods Chinese'
 'Darbar Delights South Indian' 'Spice Mess Andhra'
 'Udupi Curry House South Indian' 'Darbar Restaurant North Indian'
 'Sri Tiffins Andhra' 'Darbar Mess Pure Veg' 'Amma Kitchen Punjabi'
 'Annapurna Kitchen Family Restaurant' 'New Cafe North Indian'
 'Swagath Cafe Multicuisine' 'Ruchi Foods South Indian'
 'Annapurna Eatery Non-Veg' 'Amma Restaurant Pure Veg'
 'Amma Dhaba Non-Veg' 'Classic Curry House South Indian'
 'Annapurna Foods Non-Veg' 'Darbar Biryani Pure Veg'
 'Ruchi Dhaba Multicuisine'

In [68]:
restaurant_cuisine.update({
    'Some Restaurant Name': 'Italian',  # Replace with correct cuisine
    'Another Restaurant': 'Indian'
    # Add all missing restaurants here
})


In [69]:
final_dataset['cuisine_type'] = final_dataset['restaurant_name'].map(restaurant_cuisine)


In [72]:
#Fill in the blanks
gold_orders = final_dataset[final_dataset['membership'] == 'Gold']


In [73]:
total_gold_orders = gold_orders.shape[0]  # Number of rows/orders
print(f"Total orders placed by Gold members: {total_gold_orders}")


Total orders placed by Gold members: 4987


In [74]:
hyderabad_orders = final_dataset[final_dataset['city'] == 'Hyderabad']


In [75]:
total_revenue_hyderabad = round(hyderabad_orders['total_amount'].sum())
print(f"Total revenue from Hyderabad: ₹{total_revenue_hyderabad}")


Total revenue from Hyderabad: ₹1889367


In [76]:
distinct_users = final_dataset['user_id'].nunique()
print(f"Number of distinct users who placed at least one order: {distinct_users}")


Number of distinct users who placed at least one order: 2883


In [77]:
gold_orders = final_dataset[final_dataset['membership'] == 'Gold']



In [78]:
avg_order_value_gold = round(gold_orders['total_amount'].mean(), 2)
print(f"Average order value for Gold members: ₹{avg_order_value_gold}")


Average order value for Gold members: ₹797.15


In [79]:
# Example: Add dummy ratings randomly (for demo only)
import numpy as np
np.random.seed(0)
final_dataset['restaurant_rating'] = np.random.uniform(3.0, 5.0, size=len(final_dataset))

# Now we can count orders ≥ 4.5
high_rating_orders = final_dataset[final_dataset['restaurant_rating'] >= 4.5].shape[0]
print(high_rating_orders)


2483


In [80]:
gold_members = final_dataset[final_dataset['membership'] == 'Gold']


In [81]:
city_revenue = gold_members.groupby('city')['total_amount'].sum().reset_index()
top_city = city_revenue.sort_values(by='total_amount', ascending=False).iloc[0]['city']
print(f"Top revenue city among Gold members: {top_city}")


Top revenue city among Gold members: Chennai


In [82]:
orders_in_top_city = gold_members[gold_members['city'] == top_city].shape[0]
print(f"Number of orders in {top_city} by Gold members: {orders_in_top_city}")


Number of orders in Chennai by Gold members: 1337


In [83]:
import pandas as pd

# ----------------------------
# Step 1: Load datasets
# ----------------------------
orders = pd.read_csv("orders.csv")
users = pd.read_json("users.json")

# ----------------------------
# Step 2: Merge on user_id
# ----------------------------
# Left join to keep all orders
merged_data = pd.merge(orders, users, on='user_id', how='left')

# ----------------------------
# Step 3: Check result
# ----------------------------
print(merged_data.head())
print("\nColumns in merged dataset:", merged_data.columns)


   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name       name       city membership  
0               New Foods Chinese  User_2508  Hyderabad    Regular  
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular  
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold  
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold  
4       Royal Eatery South Indian  User_1064       Pune    Regular  

Columns in merged dataset: Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name', 'name', 'city', 'membership'],
      dtype='object')


In [89]:
import pandas as pd

# ----------------------------
# Step 1: Load the final merged dataset
# ----------------------------
final_dataset = pd.read_csv("final_food_delivery_dataset.csv")

# ----------------------------
# Step 2: Initial cuisine mapping (known restaurants)
# ----------------------------
restaurant_cuisine = {
    'Tandoori Delight': 'Indian',
    'Dragon Wok': 'Chinese',
    'Pasta House': 'Italian',
    'Mexicana': 'Mexican'
    # Add any known restaurants here
}

# ----------------------------
# Step 3: Detect unmapped restaurants
# ----------------------------
all_restaurants = final_dataset['restaurant_name'].unique()
missing_restaurants = [r for r in all_restaurants if r not in restaurant_cuisine]

print("Restaurants missing cuisine mapping:", missing_restaurants)

# ----------------------------
# Step 4: Manually assign cuisines to missing restaurants
# Replace 'Indian', 'Chinese', etc. as appropriate
# ----------------------------
for r in missing_restaurants:
    # Example assignment, update as per actual restaurant
    if 'Punjabi' in r or 'Mess' in r:
        restaurant_cuisine[r] = 'Indian'
    elif 'Chinese' in r:
        restaurant_cuisine[r] = 'Chinese'
    elif 'Italian' in r or 'Pasta' in r:
        restaurant_cuisine[r] = 'Italian'
    else:
        restaurant_cuisine[r] = 'Other'  # fallback

# ----------------------------
# Step 5: Map cuisine_type
# ----------------------------
final_dataset['cuisine_type'] = final_dataset['restaurant_name'].map(restaurant_cuisine)

# ----------------------------
# Step 6: Calculate average order value per cuisine
# ----------------------------
avg_order_value = final_dataset.groupby('cuisine_type')['total_amount'].mean().reset_index()
avg_order_value = avg_order_value.sort_values(by='total_amount', ascending=False).reset_index(drop=True)

print("\nAverage order value by cuisine:")
print(avg_order_value)

# ----------------------------
# Step 7: Top cuisine
# ----------------------------
top_cuisine = avg_order_value.iloc[0]['cuisine_type']
top_avg_value = avg_order_value.iloc[0]['total_amount']
print(f"\nCuisine with highest average order value: {top_cuisine} (₹{top_avg_value:.2f})")


Restaurants missing cuisine mapping: ['New Foods Chinese', 'Ruchi Curry House Multicuisine', 'Spice Kitchen Punjabi', 'Darbar Kitchen Non-Veg', 'Royal Eatery South Indian', 'Annapurna Tiffins South Indian', 'Royal Biryani North Indian', 'Spice Mess Punjabi', 'Ruchi Biryani Punjabi', 'Taste of Biryani Non-Veg', 'Amma Delights Family Restaurant', 'Royal Tiffins Multicuisine', 'Amma Tiffins South Indian', 'Grand Cafe Punjabi', 'Amma Biryani North Indian', 'Amma Restaurant South Indian', 'Ruchi Foods Chinese', 'Darbar Delights South Indian', 'Spice Mess Andhra', 'Udupi Curry House South Indian', 'Darbar Restaurant North Indian', 'Sri Tiffins Andhra', 'Darbar Mess Pure Veg', 'Amma Kitchen Punjabi', 'Annapurna Kitchen Family Restaurant', 'New Cafe North Indian', 'Swagath Cafe Multicuisine', 'Ruchi Foods South Indian', 'Annapurna Eatery Non-Veg', 'Amma Restaurant Pure Veg', 'Amma Dhaba Non-Veg', 'Classic Curry House South Indian', 'Annapurna Foods Non-Veg', 'Darbar Biryani Pure Veg', 'Ruchi D

In [91]:
import pandas as pd
import numpy as np

# ----------------------------
# Step 1: Load final dataset
# ----------------------------
final_dataset = pd.read_csv("final_food_delivery_dataset.csv")

# ----------------------------
# Step 2: Simulate restaurant ratings (3.0 to 5.0)
# ----------------------------
np.random.seed(0)  # for reproducibility
final_dataset['restaurant_rating'] = np.random.uniform(3.0, 5.0, size=len(final_dataset))

# ----------------------------
# Step 3: Create rating bins
# ----------------------------
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0–3.5', '3.6–4.0', '4.1–4.5', '4.6–5.0']
final_dataset['rating_range'] = pd.cut(final_dataset['restaurant_rating'], bins=bins, labels=labels, include_lowest=True)

# ----------------------------
# Step 4: Calculate total revenue by rating range
# ----------------------------
revenue_by_rating = final_dataset.groupby('rating_range')['total_amount'].sum().reset_index()
revenue_by_rating = revenue_by_rating.sort_values(by='total_amount', ascending=False).reset_index(drop=True)

print("Total revenue by restaurant rating range:")
print(revenue_by_rating)

top_range = revenue_by_rating.iloc[0]['rating_range']
print(f"\nRating range generating highest total revenue: {top_range}")


Total revenue by restaurant rating range:
  rating_range  total_amount
0      3.6–4.0    2039733.32
1      3.0–3.5    2034715.41
2      4.6–5.0    1984559.73
3      4.1–4.5    1952615.66

Rating range generating highest total revenue: 3.6–4.0


/tmp/ipython-input-1973433402.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  revenue_by_rating = final_dataset.groupby('rating_range')['total_amount'].sum().reset_index()


In [92]:
import pandas as pd

# Load the final dataset
final_dataset = pd.read_csv("final_food_delivery_dataset.csv")

# ----------------------------
# Step 1: Map restaurants to cuisines (ensure correct mapping)
# ----------------------------
restaurant_cuisine = {
    'Tandoori Delight': 'Indian',
    'Dragon Wok': 'Chinese',
    'Pasta House': 'Italian',
    'Mexicana': 'Mexican',
    'Grand Cafe Punjabi': 'Indian',
    'Grand Restaurant South Indian': 'Indian',
    'Ruchi Mess Multicuisine': 'Indian',
    'Ruchi Foods Chinese': 'Chinese'
    # Add all other restaurants from your dataset
}

final_dataset['cuisine_type'] = final_dataset['restaurant_name'].map(restaurant_cuisine)

# ----------------------------
# Step 2: Count distinct restaurants per cuisine
# ----------------------------
distinct_restaurants = final_dataset.groupby('cuisine_type')['restaurant_name'].nunique().reset_index()
distinct_restaurants.rename(columns={'restaurant_name':'num_restaurants'}, inplace=True)

# ----------------------------
# Step 3: Calculate total revenue per cuisine
# ----------------------------
total_revenue = final_dataset.groupby('cuisine_type')['total_amount'].sum().reset_index()
total_revenue.rename(columns={'total_amount':'total_revenue'}, inplace=True)

# ----------------------------
# Step 4: Merge the two metrics
# ----------------------------
cuisine_summary = pd.merge(distinct_restaurants, total_revenue, on='cuisine_type')
cuisine_summary = cuisine_summary.sort_values(by='num_restaurants')

print(cuisine_summary)


  cuisine_type  num_restaurants  total_revenue
0      Chinese                1       13045.46
1       Indian                3       82976.60


In [94]:
import pandas as pd

# Load final dataset
final_dataset = pd.read_csv("final_food_delivery_dataset.csv")

# ----------------------------
# Step 1: Ensure cuisine_type exists
# (Map restaurants to cuisines if needed)
# ----------------------------
restaurant_cuisine = {
    'Tandoori Delight': 'Indian',
    'Dragon Wok': 'Chinese',
    'Pasta House': 'Italian',
    'Mexicana': 'Mexican',
    'Grand Cafe Punjabi': 'Indian',
    'Grand Restaurant South Indian': 'Indian',
    'Ruchi Mess Multicuisine': 'Indian',
    'Ruchi Foods Chinese': 'Chinese'
    # Add other restaurants from your dataset
}

final_dataset['cuisine_type'] = final_dataset['restaurant_name'].map(restaurant_cuisine)

# ----------------------------
# Step 2: Group by membership + cuisine and sum revenue
# ----------------------------
combo_revenue = final_dataset.groupby(['membership','cuisine_type'])['total_amount'].sum().reset_index()

# Sort descending by total revenue
combo_revenue = combo_revenue.sort_values(by='total_amount', ascending=False).reset_index(drop=True)

# ----------------------------
# Step 3: Top combination
# ----------------------------
top_combo = combo_revenue.iloc[0]
print(f"Top revenue combination: {top_combo['membership']} + {top_combo['cuisine_type']} (₹{top_combo['total_amount']:.2f})")


Top revenue combination: Regular + Indian (₹45363.41)
